In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
from utils.logger_utils import setup_clean_logger, mute_external_loggers

# Set up a clean logger for your notebook
logger = setup_clean_logger("notebook", logging.INFO)
logger.info("Notebook started successfully.")

# Mute logs from sisepuede to avoid duplication
mute_external_loggers(["sisepuede"])


##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

2025-08-15 16:08:11,328 - INFO - Notebook started successfully.


In [2]:
# Save original path
original_sys_path = list(sys.path)

# Add the custom path
custom_path = os.path.abspath("../../data_processing/utils")
sys.path.append(custom_path)

# Import your module
import common_data_needs as cdn

# Revert to original sys.path
sys.path = original_sys_path

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   1084.8 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


In [3]:
%load_ext autoreload
%autoreload 2

### Initial Set up

Make sure to edit the config yaml under ssp_modeling/config_files/config.yaml

You can also create a new config yaml



In [4]:
# Set up dir paths

CURR_DIR_PATH = pathlib.Path(os.getcwd())
SSP_MODELING_DIR_PATH = CURR_DIR_PATH.parent
PROJECT_DIR_PATH = SSP_MODELING_DIR_PATH.parent
DATA_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("input_data")
RUN_OUTPUT_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("ssp_run_output")
SCENARIO_MAPPING_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("scenario_mapping")
CONFIG_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("config_files")
TRANSFORMATIONS_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("transformations")
MISC_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("misc")
STRATEGIES_DEFINITIONS_FILE_PATH = TRANSFORMATIONS_DIR_PATH.joinpath("strategy_definitions.csv")
STRATEGY_MAPPING_FILE_PATH = MISC_DIR_PATH.joinpath("strategy_mapping.yaml")

In [5]:
from ssp_transformations_handler.GeneralUtils import GeneralUtils
from ssp_transformations_handler.TransformationUtils import TransformationYamlProcessor, StrategyCSVHandler

# Initialize general utilities
g_utils = GeneralUtils()

In [6]:
# Load config file, double check your parameters are correct

YAML_FILE_PATH = os.path.join(CONFIG_DIR_PATH, "config_ccdr_ndc_2.0.yaml")
config_params = g_utils.read_yaml(YAML_FILE_PATH)

country_name = config_params['country_name']
ssp_input_file_name = config_params['ssp_input_file_name']
ssp_transformation_cw = config_params['ssp_transformation_cw']
energy_model_flag = config_params['energy_model_flag']
set_lndu_reallocation_factor_to_zero_flag = config_params['set_lndu_reallocation_factor_to_zero']

# Print config parameters
logger.info(f"Country name: {country_name}")
logger.info(f"SSP input file name: {ssp_input_file_name}")
logger.info(f"SSP transformation CW: {ssp_transformation_cw}")
logger.info(f"Energy model flag: {energy_model_flag}")
logger.info(f"Set lndu reallocation factor to zero flag: {set_lndu_reallocation_factor_to_zero_flag}")

2025-08-15 16:08:29,247 - INFO - Country name: uganda
2025-08-15 16:08:29,247 - INFO - SSP input file name: None
2025-08-15 16:08:29,247 - INFO - SSP transformation CW: ssp_uganda_transformation_cw_NDC_2.0.xlsx
2025-08-15 16:08:29,247 - INFO - Energy model flag: True
2025-08-15 16:08:29,248 - INFO - Set lndu reallocation factor to zero flag: True


In [7]:
# Set up SSP objects

# INPUT_FILE_PATH = DATA_DIR_PATH.joinpath(ssp_input_file_name)

# file_struct = SISEPUEDEFileStructure()

# matt = file_struct.model_attributes
# regions = sc.Regions(matt)
# time_periods = sc.TimePeriods(matt)

dict_ssp = cdn._setup_sisepuede_elements()

matt = dict_ssp.get("model_attributes", )
models = dict_ssp.get("models", )
regions = dict_ssp.get("regions", )
time_periods = dict_ssp.get("time_periods", )

### Making sure our input file has the correct format and correct columns
We use an example df with the complete fields and correct format to make sure our file is in the right shape

In [8]:
cdn._PATH_OUTPUTS

PosixPath('/Users/fabianfuentes/git/ssp_uganda_data/data_processing/output_data')

In [9]:
df_inputs_raw = cdn._build_from_outputs(
    (
        min(time_periods.all_years),
        max(time_periods.all_years)
    ),
    fns_exclude = ["frac_lndu_initial.csv"],
    merge_type = "outer", 
    print_info = False,
    stop_on_error = True, 
)

df_inputs_raw.head()

RuntimeError: Cannot proceed: fields 'ef_soil_c_cultivated_organic_temperate_crop_grass_tonne_per_ha', 'ef_soil_c_cultivated_organic_tropical_crop_grass_tonne_per_ha' not found.

In [ ]:
df_inputs_raw.tail()

In [ ]:
##  BUILD BASE INPUTS
# df_inputs_raw = pd.read_csv(INPUT_FILE_PATH)

# pull example data to fill in gaps
examples = SISEPUEDEExamples()
df_inputs_example = examples.input_data_frame

In [ ]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_inputs_example, df_inputs_raw)

In [ ]:
# Ensure if time_period field exist
if 'time_period' not in df_inputs_raw.columns:
    logger.info("Adding 'time_period' column to df_inputs_raw")
    df_inputs_raw = df_inputs_raw.rename(columns={'period':'time_period'})
else:
    logger.info("'time_period' column already exists in df_inputs_raw")

In [ ]:
# Fixes differences and makes sure that our df is in the correct format.
# Note: Edit this if you need more changes in your df
df_inputs_raw_complete = g_utils.add_missing_cols(df_inputs_example, df_inputs_raw.copy())
df_inputs_raw_complete = g_utils.remove_additional_cols(df_inputs_example, df_inputs_raw_complete.copy())
df_inputs_raw_complete.head()

In [ ]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_inputs_example, df_inputs_raw_complete)

In [ ]:
# Set region to country name
df_inputs_raw_complete['region'] = country_name
df_inputs_raw_complete['region'].head()

## Let's Modify the  LNDU Reallocation factor

In [ ]:
if set_lndu_reallocation_factor_to_zero_flag:
    df_inputs_raw_complete['lndu_reallocation_factor'] = 0

df_inputs_raw_complete['lndu_reallocation_factor'].mean()

In [ ]:
# Get fields with nulls
fields_with_nulls = df_inputs_raw_complete.columns[df_inputs_raw_complete.isnull().any()].tolist()
fields_with_nulls

#  Let's try building transformations using this


In [ ]:
transformers = trf.transformers.Transformers(
    {},
    attr_time_period = cdn._ATTRIBUTE_TABLE_TIME_PERIOD,
    df_input = df_inputs_raw_complete,
)

##  Instantiate some transformations. Make sure to run this cell to create the transformations folder for the first time or if you wish to overwrite

In [ ]:
# set an ouput path and instantiate
if not TRANSFORMATIONS_DIR_PATH.exists():
    trf.instantiate_default_strategy_directory(
        transformers,
        TRANSFORMATIONS_DIR_PATH,
    )
else:
    logger.info(f"Directory {TRANSFORMATIONS_DIR_PATH} already exists. Skipping instantiation.")


##  --HERE, CUSTOMIZE YOUR TRANSFORMATIONS AND STRATEGIES--

### Customizing transformations and strategies files using TransformationUtils.py classes

In [ ]:
# Generate new transformation files based on the excel mapping file. 
# Make sure to have the most updated format for the excel file, check the one used in this notebook for reference.

if ssp_transformation_cw is None:
    logger.warning("ssp_transformation_cw is not defined. Please check your config file.")
else:
    logger.info(f"Using transformation file: {ssp_transformation_cw}")
    cw_file_path = os.path.join(SCENARIO_MAPPING_DIR_PATH, ssp_transformation_cw)
    logger.info(f"Transformation file path: {cw_file_path}")
    excel_yaml_handler = TransformationYamlProcessor(scenario_mapping_excel_path=cw_file_path, yaml_dir_path=TRANSFORMATIONS_DIR_PATH)

In [ ]:
# This creates transformation yaml files for each strategy in the excel file
if ssp_transformation_cw is not None:
    logger.info("Processing YAML files...")
    excel_yaml_handler.process_yaml_files()
else:
    logger.warning("ssp_transformation_cw is not defined. Please check your config file.")

In [ ]:
# Load the transformations per strategy dictionary so we can pass it to the strategy handler
# You can also check here if the transformations in each strategy are correct

if ssp_transformation_cw is not None:
    logger.info("Loading transformations per strategy dictionary...")
    transformation_per_strategy_dict = excel_yaml_handler.get_transformations_per_strategy_dict()
    transformation_per_strategy_dict
    logger.info(f"Loaded transformations for strategies: {transformation_per_strategy_dict.keys()}")
else:
    logger.warning("No transformation handler available. Please check your config file.")

### Creating new strategies
- You can create new strategies from scratch.
- You can also update existing ones.

In [ ]:
# Creating new strategies by updating the strategy_definitions file.

if ssp_transformation_cw is not None:
    # You can edit this to add yours, as many as you want.
    csv_handler = StrategyCSVHandler(csv_file_path=STRATEGIES_DEFINITIONS_FILE_PATH, 
                                     yaml_dir_path=TRANSFORMATIONS_DIR_PATH, 
                                     yaml_mapping_file=STRATEGY_MAPPING_FILE_PATH, 
                                     transformation_per_strategy_dict=transformation_per_strategy_dict)
    
    for strategy_name in transformation_per_strategy_dict.keys():
        yaml_file_suffix = strategy_name.split('strategy_')[-1]
        csv_handler.add_strategy(strategy_group='PFLO', description='Custom Strategy', yaml_file_suffix=yaml_file_suffix)

else:
    logger.warning("No transformation handler available. Please check your config file.")


In [ ]:
# Set up the strategy codes you wish to run in ssp
strategies_to_run = [0, 6003, 6004, 6005, 6006]

## TX:LNDU:DEC_DEFORESTATION

In [ ]:

import yaml

with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NZ.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude'] = 0.9999999   


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NZ.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [ ]:

with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NDC.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude'] = 0.996


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NDC.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False) 

In [ ]:

with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NDC_2.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude'] = 0.996


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_lndu_dec_deforestation_strategy_NDC_2.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False) 

## TX:AGRC:INC_CONSERVATION_AGRICULTURE

In [ ]:
with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NZ.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude_burned'] = 0.8
data['parameters']['magnitude_removed'] = 0.8   


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NZ.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False) 

In [ ]:
with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NDC.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude_burned'] = 0.6
data['parameters']['magnitude_removed'] = 0.6   


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NDC.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False) 

In [ ]:
with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NDC_2.yaml'), 'r') as file:
    data = yaml.safe_load(file)


data['parameters']['magnitude_burned'] = 0.6
data['parameters']['magnitude_removed'] = 0.6   


with open(os.path.join(TRANSFORMATIONS_DIR_PATH, 'transformation_agrc_inc_conservation_agriculture_strategy_NDC_2.yaml'), 'w') as file:
    yaml.dump(data, file, sort_keys=False) 

### We finished adding new transformation files and strategies so lets load them back

In [ ]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    TRANSFORMATIONS_DIR_PATH,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table

In [ ]:
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
logger.info(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

In [ ]:
strategies.attribute_table

##  Build our templates
- let's use the default variable groupings for LHS

In [ ]:
# Building excel templates, make sure to include the strategies ids in the strategies attribute as well as the baseline (0)
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
    #df_trajgroup = df_vargroups,
    #include_simplex_group_as_trajgroup = True,
    strategies = strategies_to_run,
)

# Finally, load SISEPUEDE so that we can run it

In [ ]:
import sisepuede as si

In [ ]:

# timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    #id_str = f"sisepuede_run_2025-07-28T12:30:52.790396",
    initialize_as_dummy = not(energy_model_flag), # no connection to Julia is initialized if set to True
    regions = [country_name],
    strategies = strategies,
    #try_exogenous_xl_types_in_variable_specification = True,
    attribute_time_period = cdn._ATTRIBUTE_TABLE_TIME_PERIOD
)

In [ ]:
not(energy_model_flag)

In [ ]:
# This runs the model, make sure you edit key_stretegy with the strategy ids you want to execute include baseline (0)
dict_scens = {
    ssp.key_design: [0],
    ssp.key_future: [0],
    ssp.key_strategy: strategies_to_run,
}

ssp.project_scenarios(
    dict_scens,
    save_inputs = True,
    include_electricity_in_energy = energy_model_flag
)

In [ ]:
# Read input and output files
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

In [ ]:
df_out.head()

In [ ]:
def plot_field_stack(
    df,
    fields,
    dict_format,
    time_col="time_period",
    primary_id=0,
    figsize=(10, 5),
    legend_loc='center left',
    legend_bbox=(1.05, 0.5),  
    ylabel="MT Emissions CO2e",
    xlabel="Time Period",
    title=None,
):
    """
    Plots a stack plot of the selected fields for a given primary_id.

    Args:
        df (pd.DataFrame): DataFrame containing output data.
        fields (list): List of column names to plot.
        dict_format (dict): Formatting dictionary for colors.
        time_col (str): Name of the time column.
        primary_id (int): Value of primary_id to filter.
        figsize (tuple): Figure size.
        legend_loc (str): Legend location.
        legend_bbox (tuple): Legend bbox_to_anchor.
        ylabel (str): Y-axis label.
        xlabel (str): X-axis label.
        title (str): Plot title.
    """
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if title:
        ax.set_title(title)

    df_plot = df[df[ssp.key_primary].isin([primary_id])]

    fig, ax = spu.plot_stack(
        df_plot,
        fields,
        dict_formatting=dict_format,
        field_x=time_col,
        figtuple=(fig, ax),
    )

    ax.legend(loc=legend_loc, bbox_to_anchor=legend_bbox, title="Fields")
    plt.show()

In [ ]:
# Define the fields to plot and the formatting dictionary
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()

dict_format = dict(
    (k, {"color": v}) for (k, v) in
    matt.get_subsector_color_map().items()
)

In [ ]:
run_primary_ids = df_out.primary_id.unique()
run_primary_ids

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)

for single_id in run_primary_ids:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot Primary ID {single_id}"
    )

# Export Wide File (Last Mandatory Step)

In [ ]:
all_primaries = sorted(list(df_out[ssp.key_primary].unique()))

# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)




df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)



# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None


for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(dir_pkg, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        print(f"Warning: Table {tab} returned None.")


df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
# Getting the directory where the outputs are stored
ssp.file_struct.dir_out

In [ ]:
RUN_ID_OUTPUT_DIR_PATH = os.path.join(
    RUN_OUTPUT_DIR_PATH, 
    ssp.id_fs_safe
)

os.makedirs(RUN_ID_OUTPUT_DIR_PATH, exist_ok=True)

df_primary.to_csv(
    os.path.join(RUN_ID_OUTPUT_DIR_PATH, "ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(

    
    os.path.join(RUN_ID_OUTPUT_DIR_PATH, f"{ssp.id_fs_safe}.csv"),
    index = None,
    encoding = "UTF-8"
)

for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(RUN_ID_OUTPUT_DIR_PATH, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        logger.warning(f"Warning: Table {tab} returned None.")

In [ ]:
# --- Define fields  ---
fields_str = (
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_croplands:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_forests_mangroves:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_forests_primary:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_forests_secondary:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_grasslands:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_other:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_settlements:"
   "emission_co2e_co2_lndu_conversion_forests_mangroves_to_wetlands:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_croplands:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_forests_mangroves:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_forests_primary:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_forests_secondary:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_grasslands:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_other:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_settlements:"
   "emission_co2e_co2_lndu_conversion_forests_primary_to_wetlands:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_croplands:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_forests_mangroves:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_forests_primary:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_forests_secondary:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_grasslands:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_other:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_settlements:"
   "emission_co2e_co2_lndu_conversion_forests_secondary_to_wetlands:"
   "emission_co2e_ch4_lndu_wetlands"
)
subsector_emission_fields = fields_str.split(":")
subsector_emission_fields

In [ ]:
from statsmodels.tsa.filters.hp_filter import hpfilter

def hpfilter_df(df, cols, by="primary_id", x="time_period", lamb=100):
    df = df.sort_values([by, x]).copy()

    def _apply_hp(g):
        g = g.copy()
        for col in cols:
            cycle, trend = hpfilter(g[col], lamb=lamb)
            g[col] = trend.clip(lower=0)  # keep only the trend, no negatives
        return g

    return df.groupby(by, group_keys=False).apply(_apply_hp)

# Example: higher lambda → smoother trend
df_hp = hpfilter_df(
    df_out,
    subsector_emission_fields,
    by="primary_id",
    x="time_period",
    lamb=100  # try 100, 400, 1600 for different smoothness
)

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)

for single_id in run_primary_ids:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot (HP filtered) {single_id}" 
    )

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)

for single_id in run_primary_ids:

    plot_field_stack(
        df_hp,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot (HP filtered) {single_id}" 
    )

In [ ]:
df_out[subsector_emission_fields] = df_hp[subsector_emission_fields]

In [ ]:
# --- Define fields  ---
fields_str = (
   "emission_co2e_co2_entc_generation_pp_biogas:"
   "emission_co2e_co2_entc_generation_pp_biomass:"
   "emission_co2e_co2_entc_generation_pp_coal:"
   "emission_co2e_co2_entc_generation_pp_coal_ccs:"
   "emission_co2e_co2_entc_generation_pp_gas:"
   "emission_co2e_co2_entc_generation_pp_gas_ccs:"
   "emission_co2e_co2_entc_generation_pp_geothermal:"
   "emission_co2e_co2_entc_generation_pp_hydropower:"
   "emission_co2e_co2_entc_generation_pp_nuclear:"
   "emission_co2e_co2_entc_generation_pp_ocean:"
   "emission_co2e_co2_entc_generation_pp_oil:"
   "emission_co2e_co2_entc_generation_pp_solar:"
   "emission_co2e_co2_entc_generation_pp_waste_incineration:"
   "emission_co2e_co2_entc_generation_pp_wind:"
   "emission_co2e_n2o_entc_generation_pp_biogas:"
   "emission_co2e_n2o_entc_generation_pp_biomass:"
   "emission_co2e_n2o_entc_generation_pp_coal:"
   "emission_co2e_n2o_entc_generation_pp_coal_ccs:"
   "emission_co2e_n2o_entc_generation_pp_gas:"
   "emission_co2e_n2o_entc_generation_pp_gas_ccs:"
   "emission_co2e_n2o_entc_generation_pp_geothermal:"
   "emission_co2e_n2o_entc_generation_pp_hydropower:"
   "emission_co2e_n2o_entc_generation_pp_nuclear:"
   "emission_co2e_n2o_entc_generation_pp_ocean:"
   "emission_co2e_n2o_entc_generation_pp_oil:"
   "emission_co2e_n2o_entc_generation_pp_solar:"
   "emission_co2e_n2o_entc_generation_pp_waste_incineration:"
   "emission_co2e_n2o_entc_generation_pp_wind:"
   "emission_co2e_ch4_entc_generation_pp_biogas:"
   "emission_co2e_ch4_entc_generation_pp_biomass:"
   "emission_co2e_ch4_entc_generation_pp_coal:"
   "emission_co2e_ch4_entc_generation_pp_coal_ccs:"
   "emission_co2e_ch4_entc_generation_pp_gas:"
   "emission_co2e_ch4_entc_generation_pp_gas_ccs:"
   "emission_co2e_ch4_entc_generation_pp_geothermal:"
   "emission_co2e_ch4_entc_generation_pp_hydropower:"
   "emission_co2e_ch4_entc_generation_pp_nuclear:"
   "emission_co2e_ch4_entc_generation_pp_ocean:"
   "emission_co2e_ch4_entc_generation_pp_oil:"
   "emission_co2e_ch4_entc_generation_pp_solar:"
   "emission_co2e_ch4_entc_generation_pp_waste_incineration:"
   "emission_co2e_ch4_entc_generation_pp_wind"
)
subsector_emission_fields = fields_str.split(":")
subsector_emission_fields

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)
for single_id in run_primary_ids:

    plot_field_stack(
        df_hp,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot (HP filtered) {single_id}" 
    )

In [ ]:
# Define cut year where you want the flat projection to start
cut_year = 2

for pid in run_primary_ids:
    print(pid)
    # Mask for the future part
    mask_future = (df_out["primary_id"] == pid) & (df_out["time_period"] >= cut_year)

    # Get the last historical values before the cut
    last_vals = df_out.loc[
        (df_out["primary_id"] == pid) & (df_out["time_period"] < cut_year),
        subsector_emission_fields
    ].iloc[-1]

    # Assign the last values to all future rows
    df_out.loc[mask_future, subsector_emission_fields] = last_vals.values

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)
for single_id in run_primary_ids:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot {single_id}" 
    )

In [ ]:
# --- Define fields  ---
fields_str = (
    "emission_co2e_co2_fgtv_fuel_coal:"
    "emission_co2e_co2_fgtv_fuel_natural_gas:"
    "emission_co2e_co2_fgtv_fuel_oil:"
    "emission_co2e_co2_entc_fuel_mining_and_extraction_me_coal:"
    "emission_co2e_co2_entc_fuel_mining_and_extraction_me_crude:"
    "emission_co2e_co2_entc_fuel_mining_and_extraction_me_natural_gas:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_ammonia_production:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_hydrogen_electrolysis:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_hydrogen_gasification:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_hydrogen_reformation:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_hydrogen_reformation_ccs:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_natural_gas:"
    "emission_co2e_co2_entc_processing_and_refinement_fp_petroleum_refinement:"
    "emission_co2e_ch4_fgtv_fuel_coal:"
    "emission_co2e_ch4_fgtv_fuel_natural_gas:"
    "emission_co2e_ch4_fgtv_fuel_oil:"
    "emission_co2e_ch4_entc_fuel_mining_and_extraction_me_coal:"
    "emission_co2e_ch4_entc_fuel_mining_and_extraction_me_crude:"
    "emission_co2e_ch4_entc_fuel_mining_and_extraction_me_natural_gas:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_ammonia_production:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_hydrogen_electrolysis:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_hydrogen_gasification:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_hydrogen_reformation:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_hydrogen_reformation_ccs:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_natural_gas:"
    "emission_co2e_ch4_entc_processing_and_refinement_fp_petroleum_refinement:"
    "emission_co2e_n2o_fgtv_fuel_coal:"
    "emission_co2e_n2o_fgtv_fuel_natural_gas:"
    "emission_co2e_n2o_fgtv_fuel_oil:"
    "emission_co2e_n2o_entc_fuel_mining_and_extraction_me_coal:"
    "emission_co2e_n2o_entc_fuel_mining_and_extraction_me_crude:"
    "emission_co2e_n2o_entc_fuel_mining_and_extraction_me_natural_gas:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_ammonia_production:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_hydrogen_electrolysis:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_hydrogen_gasification:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_hydrogen_reformation:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_hydrogen_reformation_ccs:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_natural_gas:"
    "emission_co2e_n2o_entc_processing_and_refinement_fp_petroleum_refinement"
)
subsector_emission_fields = fields_str.split(":")
subsector_emission_fields

In [ ]:
from statsmodels.tsa.filters.hp_filter import hpfilter

def hpfilter_df(df, cols, by="primary_id", x="time_period", lamb=100):
    df = df.sort_values([by, x]).copy()

    def _apply_hp(g):
        g = g.copy()
        for col in cols:
            cycle, trend = hpfilter(g[col], lamb=lamb)
            g[col] = trend.clip(lower=0)  # keep only the trend, no negatives
        return g

    return df.groupby(by, group_keys=False).apply(_apply_hp)

# Example: higher lambda → smoother trend
df_hp = hpfilter_df(
    df_out,
    subsector_emission_fields,
    by="primary_id",
    x="time_period",
    lamb=100  # try 100, 400, 1600 for different smoothness
)

plot_field_stack(
    df_hp,
    subsector_emission_fields,
    dict_format,
    primary_id=0,
    title="Emissions Stack Plot (HP filtered)"
)


In [ ]:
# Define cut year where you want the flat projection to start
cut_year = 2

for pid in run_primary_ids:
    print(pid)
    # Mask for the future part
    mask_future = (df_out["primary_id"] == pid) & (df_out["time_period"] >= cut_year)

    # Get the last historical values before the cut
    last_vals = df_out.loc[
        (df_out["primary_id"] == pid) & (df_out["time_period"] < cut_year),
        subsector_emission_fields
    ].iloc[-1]

    # Assign the last values to all future rows
    df_out.loc[mask_future, subsector_emission_fields] = last_vals.values

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)
for single_id in run_primary_ids:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot {single_id}" 
    )

In [ ]:
# --- Define fields  ---
fields_str = (
   "emission_co2e_co2_scoe_commercial_municipal:emission_co2e_co2_scoe_other_se:emission_co2e_co2_scoe_residential:"
   "emission_co2e_ch4_scoe_commercial_municipal:emission_co2e_ch4_scoe_other_se:emission_co2e_ch4_scoe_residential:"
   "emission_co2e_n2o_scoe_commercial_municipal:emission_co2e_n2o_scoe_other_se:emission_co2e_n2o_scoe_residential"

)
subsector_emission_fields = fields_str.split(":")
subsector_emission_fields

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)
for single_id in run_primary_ids:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot {single_id}" 
    )

In [ ]:
# Define cut year where you want the flat projection to start
cut_year = 3
pid = run_primary_ids[0]
print(run_primary_ids[0])
# Mask for the future part
mask_future = (df_out["primary_id"] == pid) & (df_out["time_period"] >= cut_year)

# Get the last historical values before the cut
last_vals = df_out.loc[
    (df_out["primary_id"] == pid) & (df_out["time_period"] < cut_year),
    subsector_emission_fields
].iloc[-1]

# Assign the last values to all future rows
df_out.loc[mask_future, subsector_emission_fields] = last_vals.values

In [ ]:
# Define cut year where you want the flat projection to start
cut_year = 3
pid = run_primary_ids[4]
print(run_primary_ids[4])
# Mask for the future part
mask_future = (df_out["primary_id"] == pid) & (df_out["time_period"] >= cut_year)

# Get the last historical values before the cut
last_vals = df_out.loc[
    (df_out["primary_id"] == pid) & (df_out["time_period"] < cut_year),
    subsector_emission_fields
].iloc[-1]

# Assign the last values to all future rows
df_out.loc[mask_future, subsector_emission_fields] = last_vals.values

In [ ]:
# Example: higher lambda → smoother trend
df_hp = hpfilter_df(
    df_out,
    subsector_emission_fields,
    by="primary_id",
    x="time_period",
    lamb=1600  # try 100, 400, 1600 for different smoothness
)

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)

for single_id in run_primary_ids:

    plot_field_stack(
        df_hp,
        subsector_emission_fields,
        dict_format,
        primary_id=single_id,
        title=f"Emissions Stack Plot (HP filtered) {single_id}"
    )

In [ ]:
df_out[subsector_emission_fields] = df_hp[subsector_emission_fields]